# Performance

This notebook covers some of the most common performance tips for Julia.

In [ ]:
using DataFrames, CSV

In [ ]:
sensors = CSV.read("data/bay_area_freeways.csv", DataFrame);

## Loops are okay

One of the fundamental control structures of computer programming is the loop, which allows executing code repeatedly.

If you're used to programming in other dynamic languages (R, Python, etc.) you've likely been told to avoid using loops at all costs. In those languages, loops are very slow, because they do not know the types of the data structures being operated on in the loops. Instead, you are told to "vectorize" - use operations that operate on a whole array of data at once (most `np.` functions in Python, `vapply` in R, etc.). These functions are implemented in efficient native code written in C or C++, and thus avoid the overhead of a loop. In Julia, vectorized operations are available; in fact, any function or operator can be vectorized by adding a .

If you're used to programming in static languages (C, Fortran, Java, etc.), using loops is likely second nature. These languages do not suffer from the problems described above. Julia is a mix of static and dynamic, but loops are not in general slow in Julia.

In fact, using a loop in Julia is often more efficient than using vectorized operations in either Julia or other languages. In the code below, we're counting up the number of periods where traffic sensors had speeds above or below 30 mph. Using a vectorized operation, you might create two arrays, with ones for each observation where speed is above or below 30 mph, respectively, then sum them. This requires 

That said, if vectorized operations are more readable, there's nothing wrong with using them in Julia - vectorization is also fast in Julia, and the gains from using a loop are marginal and you should only try to optimize for speed over readability if speed really is an issue.

In [ ]:
@time begin
    count_under30mph = 0
    count_over30mph = 0
    
    for value in sensors.avg_speed_mph
        if value > 30
            count_over30mph += 1
        elseif value ≤ 30
            count_under30mph += 1
        end
    end
end

## Use functions so Julia's type system can optimize performance-critical code

In [ ]:
function compute_sensor_values(data)
    count_under30mph = 0
    count_over30mph = 0
    
    for value in data.avg_speed_mph
        if value > 30
            count_over30mph += 1
        elseif value ≤ 30
            count_under30mph += 1
        end
    end
    
    return count_under30mph, count_over30mph
end

In [ ]:
@time compute_sensor_values(sensors)

## Use @code_warntype to see performance problems

In [ ]:
@code_warntype compute_sensor_values(sensors)

## Fix performance problems by changing function parameters

The problem is that DataFrames can contain any type of column, so Julia can't know that `avg_speed_mph` will always be a floaring point number. There are several ways to fix this; the most straightforward is to pass the column as an argument to the function, rather than the entire data frame. Each column has a defined type.

In [ ]:
function compute_sensor_values_column(column)
    count_under30mph = 0
    count_over30mph = 0
    
    for value in column
        if value > 30
            count_over30mph += 1
        elseif value ≤ 30
            count_under30mph += 1
        end
    end
    
    return count_under30mph, count_over30mph
end

In [ ]:
@time compute_sensor_values_column(sensors.avg_speed_mph)

In [ ]:
@code_warntype compute_sensor_values_column(sensors.avg_speed_mph)

## Looping over the rows of a data frame

Sometimes you want to loop over the rows of a data frame, not just a single column. Because the column types are unknown, this can be quite slow, even inside a function. For example, suppose we want to compute how often a sensor is actually detecting a car, not the estimated amount of time based on filling in gaps using imputation.

In [ ]:
function loop_over_rows(data)
    total = 0
    count = 0
    
    for row in eachrow(data)
        total += row.pct_obs / 100 * row.avg_occ
        count += 1
    end
    
    total / count
end

In [ ]:
@time loop_over_rows(sensors)

In [ ]:
@code_warntype loop_over_rows(sensors)

In [ ]:
function stable_loop_over_rows(itr)
    total = 0
    count = 0
    
    for row in itr
        total += row.pct_obs / 100 * row.avg_occ
        count += 1
    end
    
    total / count
end

In [ ]:
@time stable_loop_over_rows(Tables.namedtupleiterator(sensors))

In [ ]:
@code_warntype stable_loop_over_rows(Tables.namedtupleiterator(sensors))

In [ ]:
function stable_loop_over_rows(itr)
    total = 0.0
    count = 0
    
    for row in itr
        total += row.pct_obs / 100 * row.avg_occ
        count += 1
    end
    
    total / count
end

In [ ]:
@time stable_loop_over_rows(Tables.namedtupleiterator(sensors))

In [ ]:
@code_warntype stable_loop_over_rows(Tables.namedtupleiterator(sensors))

## Shut down the kernel!

When done, choose Kernel -> Shut Down Kernel to free memory for the next exercise.